# Load data

In [13]:
import pandas as pd

df_dataset = pd.read_csv("preprocessed_dataset.csv",index_col=0).drop(columns=["1d_pct_price_var","10d_pct_price_var"])

# Train - test separation

In [14]:
df_train = df_dataset.iloc[:-900]
df_test = df_dataset.iloc[-900:]

x_train = df_train.drop(columns=["5d_pct_price_var","observation_date"])
y_train = df_train["5d_pct_price_var"]
x_test = df_test.drop(columns=["5d_pct_price_var","observation_date"])
y_test = df_test["5d_pct_price_var"]

# Model Hyperparameters Selection (sul mio pc ci mette 10 minuti)

In [15]:
# !pip install lightgbm
from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)


estimator = LGBMRegressor(verbose=-1)
param_grid = {
    "n_estimators": [5, 10, 50,100,500],
    "max_depth": [-1,5,10],
    "min_child_samples": [5, 10, 20, 50, 100], 
    "learning_rate": [0.001, 0.005,0.01, 0.02,0.05],
    "lambda_l2":[1,10,100]
}

grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=param_grid,
    scoring='r2',
    cv=4,
    verbose=1
)

grid_search.fit(x_train, y_train)

print("LGBM")
print("Best score:", grid_search.best_score_)
print("Best parameters:", grid_search.best_params_)


Fitting 4 folds for each of 1125 candidates, totalling 4500 fits
LGBM
Best score: -0.007543359542482142
Best parameters: {'lambda_l2': 10, 'learning_rate': 0.05, 'max_depth': 10, 'min_child_samples': 10, 'n_estimators': 5}


In [16]:
grid_search.score(x_test,y_test)

-0.018004683624377682

In [17]:
import numpy as np

def sign_accuracy(y_true, y_pred):
    return np.mean(np.sign(y_true) == np.sign(y_pred))

print("Train dataset performance: ",sign_accuracy(y_train,grid_search.predict(x_train)))
print("Test dataset performance: ",sign_accuracy(y_test,grid_search.predict(x_test)))

Train dataset performance:  0.6491666666666667
Test dataset performance:  0.52


In [18]:
from scipy.stats import binom_test

# Example inputs
n_total = 900        # Total number of samples (adjust to your case)
n_correct = int(0.52 * n_total)  # Number of correct sign predictions (52% of total)

# Perform one-sided binomial test (greater than 50%)
p_value = binom_test(n_correct, n_total, p=0.5, alternative='greater')

print(f"Sign Accuracy: {n_correct / n_total:.2%}")
print(f"P-value: {p_value:.5f}")

Sign Accuracy: 52.00%
P-value: 0.12166


C:\Users\volpe\AppData\Local\Temp\ipykernel_17520\3518836300.py:8: DeprecationWarning: 'binom_test' is deprecated in favour of 'binomtest' from version 1.7.0 and will be removed in Scipy 1.12.0.
  p_value = binom_test(n_correct, n_total, p=0.5, alternative='greater')
